In [2]:
import tensorflow as tf
import numpy as np

## AlexNet网络结构

![alexnet](./alex.png)

In [3]:
def conv_layer(x, filters, n_filters, stride, name, padding='same', groups=1):
    input_channels = int(x.get_shape()[-1])
    
    conv = lambda i, k: tf.nn.conv2d(i, k, strides=stride, padding=padding)
    
    with tf.variable_scope(name):
        weights = tf.get_variable('weights', shape=[filters[1], filters[2], input_channels/groups, n_filters])
        biases = tf.get_variable('biases', shape=[n_filters])
    
    if groups == 1:
        layer = conv(x, weights)
    else:
        input_groups = tf.split(axis=3, num_or_size_splits=groups, value=x)    
        weights_groups = tf.split(axis=3, num_or_size_splits=groups, value=weights)
        output_groups = [conv(i, k) for i, k in zip(input_groups, weights_groups)]
        layer = tf.concat(axis=3, values=output_groups)
    
    layer = tf.reshape(tf.nn.bias_add(layer, biases), tf.shape(layer))
    layer = tf.nn.relu(layer, name=scope.name)
    
    return layer

In [4]:
def fc_layer(x, in_size, out_size, name):
    with tf.variable_scope(name):
        weights = tf.get_variable('weights', shape=[in_size, out_size], trainable=True)
        biases = tf.get_variable('biases', shape=[out_size], trainable=True)
        
    layer = tf.nn.xw_plus_b(x, weights, biases, name=scope.name)
    
    return layer

In [5]:
def max_pool_layer(x, filters, stride, name, padding='SAME'):
    return tf.nn.max_pool(x, ksize=filters, strides=stride, padding=padding, name=name)

In [6]:
def lrn(x, radius, alpha, beta, name, bias=1.0):
    return tf.nn.local_response_normalization(x, depth_radius=radius, alpha=alpha, beta=beta, bias=bias, name=name)

In [7]:
def dropout(x, keep_prob):
    return tf.nn.dropout(x, keep_prob)

In [9]:
keep_prob = 0.5
n_class = 1000

In [10]:
def alex_net(X):
    # 1st layer
    conv1 = conv_layer(X, [1, 11, 11, 1], 96, [1, 4, 4, 1], padding='VALID', name='conv1')
    norm1 = lrn(conv1, 2, 2e-5, 0.75, name='norm1')
    pool1 = max_pool_layer(norm1, [1, 3, 3, 1], [1, 2, 2, 1], padding='VALID', name='pool1')
    
    # 2nd layer
    conv2 = conv_layer(pool1, [1, 5, 5, 1], 256, [1, 1, 1, 1], groups=2, name='conv2')
    norm2 = lrn(conv2, 2, 2e-5, 0.75, name='norm2')
    pool2 = max_pool_layer(norm2, [1, 3, 3, 1], [1, 2, 2, 1], padding='VALID', name='pool2')
    
    # 3rd layer
    conv3 = conv_layer(pool2, [1, 3, 3, 1], 384, [1, 1, 1, 1], name='conv3')
    
    # 4th layer
    conv4 = conv_layer(conv3, [1, 3, 3, 1], 384, [1, 1, 1, 1], groups=2, name='conv4')
    
    # 5th layer
    conv5 = conv_layer(conv4, [1, 3, 3, 1], 256, [1, 1, 1, 1], groups=2, name='conv5')
    pool5 = max_pool_layer(conv5, [1, 3, 3, 1], [1, 2, 2, 1], padding='VALID', name='pool5')
    
    # 6th layer
    flatten = tf.reshape(pool5, [-1, 6*6*256])
    fc6 = fc_layer(flatten, 6*6*256, 4096, name='fc6')
    relu6 = tf.nn.relu(fc6)
    dropout6 = dropout(relu6, keep_prob)
    
    # 7th layer
    fc7 = fc_layer(dropout6, 4096, 4096, name='fc7')
    relu7 = tf.nn.relu(fc7)
    dropout7 = dropout(relu7, keep_prob)
    
    # 8th layer
    fc8 = fc_layer(dropout7, 4097, n_classes, name='fc8')
    
    return fc8